In [2]:
%tensorflow_version 2.x
from google.colab import drive
drive.mount("/content/drive/", force_remount=True)
import sys
sys.path.append("/content/drive/My Drive/RankNet")

Mounted at /content/drive/


In [3]:
from indycar.model.deepartfve.dataset.time_series import RecordTs
from indycar.model.deepartfve.model.lstm import DeepAR

In [5]:
#import os
#os.environ["CUDA_VISIBLE_DEVICES"]="0"
tf.compat.v1.disable_eager_execution()

def get_sample_prediction_gaussian(sample, fn):
    sample = np.array(sample).reshape(1, _seq_len, 1)
    output = fn([sample])
    samples = []
    for mu,sigma in zip(output[0].reshape(_seq_len), output[1].reshape(_seq_len)):
        samples.append(normal(loc=mu, scale=np.sqrt(sigma), size=1)[0])
    return np.array(samples)

def get_sample_prediction(sample, model, verbose = False):
    sample = np.array(sample).reshape(1, _seq_len, -1)
    output = model.predict([sample])
    
    output2 = np.zeros((_seq_len, len(output)))
    for idx, x in enumerate(output):
        output2[:,idx] = x.reshape(_seq_len)
        
    #output2 = np.array(output).reshape(_seq_len, -1)
    if verbose:
        print('output.shape=',[len(x) for x in output])
        print('output:', output)
        print('output2.shape=', output2.shape)
        print('output2:', output2)
    
    samples = []
    #for theta in zip(output[0].reshape(_seq_len), output[1].reshape(_seq_len)):
    for theta in output2:
        samples.append(model.get_sample(theta))
    return np.array(samples)


def predict(model):
    #batch = ts.next_batch(_batch_size, _seq_len)
    batch = ts.next_batch(-1, _seq_len)

    #get_sample_prediction(batch[0], model, verbose=True)
    #get_sample_prediction(batch[0], model, verbose=True)
    
    ress = []
    for i in tqdm.tqdm(range(300)):
        #ress.append(get_sample_prediction(batch[0], model.predict_theta_from_input))
        ress.append(get_sample_prediction(batch[0], model))

    res_df = pd.DataFrame(ress).T
    tot_res = res_df

    plt.plot(batch[1].reshape(_seq_len), linewidth=6)
    tot_res['mu'] = tot_res.apply(lambda x: np.mean(x), axis=1)
    tot_res['upper'] = tot_res.apply(lambda x: np.mean(x) + np.std(x), axis=1)
    tot_res['lower'] = tot_res.apply(lambda x: np.mean(x) - np.std(x), axis=1)
    tot_res['two_upper'] = tot_res.apply(lambda x: np.mean(x) + 2*np.std(x), axis=1)
    tot_res['two_lower'] = tot_res.apply(lambda x: np.mean(x) - 2*np.std(x), axis=1)

    plt.plot(tot_res.mu, 'bo')
    plt.plot(tot_res.mu, linewidth=2)
    plt.fill_between(x = tot_res.index, y1=tot_res.lower, y2=tot_res.upper, alpha=0.5)
    plt.fill_between(x = tot_res.index, y1=tot_res.two_lower, y2=tot_res.two_upper, alpha=0.5)
    plt.title('Prediction uncertainty')
    
    return batch, tot_res

import tensorflow as tf
tf.config.experimental_run_functions_eagerly(False)
#tf.config.experimental_run_functions_eagerly(True)

#ts = RecordTs('savedata_drank_e1.pickle')
ts = RecordTs('/content/drive/My Drive/RankNet/savedata_drank_e1.pickle')
_context_len = 40
_prediction_len = 2
_batch_size = 32
_seq_len = _context_len + _prediction_len


# train
model3 = DeepAR(ts, epochs=10, distribution='StudentT', with_custom_nn_structure=DeepAR.encoder_decoder)
model3.fit(context_len=_context_len, prediction_len=_prediction_len, input_dim = 33)


# prediction
#batch, df = predict(model3)

dataset shape: (32, 42, 33)
target shape: (32, 42)
y_true.shape=(None, None, None)
Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 42, 33)]          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 42, 40)            11840     
_________________________________________________________________
lstm_3 (LSTM)                (None, 42, 40)            12960     
_________________________________________________________________
main_output (StudentTLayer)  [(None, 42, 1), (None, 42 123       
Total params: 24,923
Trainable params: 24,923
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
50/50 [==============================] - 2s 34ms/step - batch: 24.5000 - size: 32.0000 - loss: 4.6726
Epoch 2/10
50/50 [==============================] - 

In [6]:
# prediction
batch, df = predict(model3)

NameError: ignored